In [ ]:
def wave_extraction(data, cutoff=0.01):
    """
    Extracts waves from price data using linear regression to fit two lines
    :param data: DataFrame with 'High' and 'Low' columns
    :param cutoff: loss cutoff to determine the end of a wave
    :return: list of waves
    """
    waves = []
    current_wave = []
    reg_high = LinearRegression()
    reg_low = LinearRegression()

    for i in range(len(data)):
        current_wave.append(data.iloc[i])
        if len(current_wave) > 1:
            X = np.arange(len(current_wave)).reshape(-1, 1)
            y_high = np.array([candle['High'] for candle in current_wave]).reshape(-1, 1)
            y_low = np.array([candle['Low'] for candle in current_wave]).reshape(-1, 1)

            reg_high.fit(X, y_high)
            reg_low.fit(X, y_low)

            loss_high = np.mean((reg_high.predict(X) - y_high) ** 2)
            loss_low = np.mean((reg_low.predict(X) - y_low) ** 2)

            if (loss_high + loss_low) / 2 > cutoff:
                waves.append(current_wave[:-1])
                current_wave = [current_wave[-1]]

    waves.append(current_wave)
    return waves

# Example usage
data = pd.read_csv('price_data.csv')
waves = wave_extraction(data)


NameError: name 'pd' is not defined

In [ ]:
def prepare_data(waves):
    """
    Prepare the data by extracting features from the waves
    :param waves: list of waves
    :return: DataFrame of features
    """
    features = []

    for wave in waves:
        num_candles = len(wave)
        high_last = wave[-1]['High']
        low_last = wave[-1]['Low']
        avg_high_low = np.mean([(candle['High'] + candle['Low']) / 2 for candle in wave])
        slope_upper = (wave[-1]['High'] - wave[0]['High']) / num_candles
        slope_lower = (wave[-1]['Low'] - wave[0]['Low']) / num_candles

        indicators = {
            'MACD': talib.MACD(np.array([candle['Close'] for candle in wave]))[0][-1],
            'EMA': talib.EMA(np.array([candle['Close'] for candle in wave]))[-1],
            'SMA': talib.SMA(np.array([candle['Close'] for candle in wave]))[-1],
            'ADX': talib.ADX(np.array([candle['High'] for candle in wave]), np.array([candle['Low'] for candle in wave]), np.array([candle['Close'] for candle in wave]))[-1],
            'RSI': talib.RSI(np.array([candle['Close'] for candle in wave]))[-1],
            'Stochastic': talib.STOCH(np.array([candle['High'] for candle in wave]), np.array([candle['Low'] for candle in wave]), np.array([candle['Close'] for candle in wave]))[0][-1],
            'WilliamsR': talib.WILLR(np.array([candle['High'] for candle in wave]), np.array([candle['Low'] for candle in wave]), np.array([candle['Close'] for candle in wave]))[-1],
            'BollingerBands': talib.BBANDS(np.array([candle['Close'] for candle in wave]))[0][-1],
            'KeltnerChannel': talib.KAMA(np.array([candle['Close'] for candle in wave]))[-1]
        }

        feature_row = [num_candles, high_last, low_last, avg_high_low, slope_upper, slope_lower] + list(indicators.values())
        features.append(feature_row)

    columns = ['num_candles', 'high_last', 'low_last', 'avg_high_low', 'slope_upper', 'slope_lower',
               'MACD', 'EMA', 'SMA', 'ADX', 'RSI', 'Stochastic', 'WilliamsR', 'BollingerBands', 'KeltnerChannel']

    return pd.DataFrame(features, columns=columns)

# Example usage
feature_data = prepare_data(waves)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv1D, Dense, Dropout, Input, Add, Activation, Concatenate, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.activations import selu
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import talib

# Define the Dilated Causal Convolutional Layer with Skip Connections
class DilatedCausalConv(Layer):
    def __init__(self, filters, kernel_size, dilation_rate):
        super(DilatedCausalConv, self).__init__()
        self.conv = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal', activation=selu)

    def call(self, inputs):
        x = self.conv(inputs)
        return x

# Define the Att-DCNN model
def build_att_dcnn(input_shape, num_filters, kernel_size, num_dilations, dropout_rates):
    inputs = Input(shape=input_shape)
    x = inputs
    skip_connections = []

    for i in range(num_dilations):
        x = DilatedCausalConv(num_filters, kernel_size, dilation_rate=2**i)(x)
        skip_connections.append(x)
        x = Dropout(dropout_rates[i])(x)

    x = Add()(skip_connections)
    x = Conv1D(100, 1, activation=selu)(x)
    x = Dropout(0.5)(x)  # Final dropout before FC layer
    outputs = Dense(100, activation='selu')(x)
    return Model(inputs, outputs)

# Define the Attention Mechanism for Event Embeddings
def self_attention(x, units):
    attention = Dense(units, activation='tanh')(x)
    attention = Dense(units, activation='softmax')(attention)
    context = attention * x
    return tf.reduce_sum(context, axis=1)

# Define the Att-biNTN model
def build_att_bintn(input_shape, num_heads, head_size):
    inputs = Input(shape=input_shape)
    x = inputs

    for _ in range(num_heads):
        x = self_attention(x, head_size)

    x = Dense(100, activation='selu')(x)
    return Model(inputs, x)

# Combine the models into Att-DiCE
def build_att_dice(financial_input_shape, event_input_shape, num_filters, kernel_size, num_dilations, dropout_rates, num_heads, head_size):
    financial_inputs = Input(shape=financial_input_shape)
    event_inputs = Input(shape=event_input_shape)

    att_dcnn = build_att_dcnn(financial_input_shape, num_filters, kernel_size, num_dilations, dropout_rates)(financial_inputs)
    att_bintn = build_att_bintn(event_input_shape, num_heads, head_size)(event_inputs)

    combined = Concatenate()([att_dcnn, att_bintn])
    outputs = Dense(1, activation='sigmoid')(combined)

    return Model([financial_inputs, event_inputs], outputs)

# Define the parameters
financial_input_shape = (40, 10)  # Example shape
event_input_shape = (40, 100)  # Example shape
num_filters = 64
kernel_size = 3
num_dilations = 4
dropout_rates = [0.2, 0.3, 0.4, 0.5]
num_heads = 3
head_size = 100

# Build the model
att_dice_model = build_att_dice(financial_input_shape, event_input_shape, num_filters, kernel_size, num_dilations, dropout_rates, num_heads, head_size)

# Compile the model
att_dice_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
att_dice_model.summary()
